In [1]:
import weaviate

connection_params = {
    "url": "http://localhost:8080",
}

# Initialize the client with connection parameters
weaviate_client = weaviate.Client(
    url=connection_params['url'],
    timeout_config=(30, 30)  # Set the connect and read timeout values in seconds
)


C:\Users\grc\python_test\fcc-gpt-course\cuda\lib\site-packages\weaviate\warnings.py:162: DeprecationWarning: Dep016: Python client v3 `weaviate.Client(...)` connections and methods are deprecated. Update
            your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.

            For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
            For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration
            
  warnings.warn(
C:\Users\grc\python_test\fcc-gpt-course\cuda\lib\site-packages\weaviate\warnings.py:124: DeprecationWarning: Dep004: You are connected to Weaviate 1.15.2.
            Consider upgrading to the latest version. See https://www.weaviate.io/developers/weaviate for details.
  warnings.warn(


In [3]:
schema = {
    "class": "TaxCode",
    "vectorizer": "text2vec-contextionary",  # Use a supported vectorizer
    "properties": [
        {
            "name": "content",
            "dataType": ["text"],
        },
        {
            "name": "chunk_id",
            "dataType": ["string"],
        }
    ]
}

try:
    weaviate_client.schema.create_class(schema)
    print("TaxCode class created successfully.")
except weaviate.exceptions.UnexpectedStatusCodeException as e:
    if "already exists" in str(e):
        print("TaxCode class already exists in Weaviate.")
    else:
        print(f"Failed to create class: {e}")


TaxCode class created successfully.


In [4]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text()
    return text

def chunk_text(text, chunk_size=1000):
    chunks = []
    for i in range(0, len(text), chunk_size):
        chunks.append(text[i:i + chunk_size])
    return chunks

pdf_path = r"C:\Users\grc\Downloads\books_for_Train\Tax_code\usc26@118-64.pdf"
tax_code_text = extract_text_from_pdf(pdf_path)
chunks = chunk_text(tax_code_text)

In [5]:
for i, chunk in enumerate(chunks):
    weaviate_client.data_object.create(
        {
            "content": chunk,
            "chunk_id": str(i)
        },
        "TaxCode"
    )

In [22]:
from dspy.retrieve.weaviate_rm import WeaviateRM
retriever_model = WeaviateRM("WeaviateTaxman", weaviate_client=weaviate_client, k=10)

def retrieve_chunks(query, retriever_model, k=5):
    results = retriever_model.retrieve(query, top_k=k)
    return [result['text'] for result in results]

import dspy 
llama_3_ollama = dspy.OllamaLocal(model='dolphin-llama3')

# Configure dspy settings
dspy.settings.configure(lm=llama_3_ollama, rm=retriever_model)


class WeaviateRM:
    def __init__(self, class_name, weaviate_client, k=10):
        self.class_name = class_name
        self.client = weaviate_client
        self.k = k

    def retrieve(self, query, top_k=None):
        if top_k is None:
            top_k = self.k
        response = self.client.query.get(self.class_name, ["content"]).with_near_text({"concepts": [query]}).with_limit(top_k).do()
        return response["data"]["Get"][self.class_name]

In [29]:
from typing import List 

class TaxCodeRAG(dspy.Module):
    def __init__(self, retriever_model):
        super().__init__()
        self.retriever_model = retriever_model
        
        class TaxExpertSignature(dspy.Signature):
            """You are a tax expert. Answer the question based on the given context."""
            context: List[str] = dspy.InputField()
            question: str = dspy.InputField()
            answer: str = dspy.OutputField()

        self.generate = dspy.ChainOfThought(TaxExpertSignature)
    

    def forward(self, question):
        # Retrieve context from Weaviate
        context_results = self.retriever_model.retrieve(question, top_k=5)
        context = [result['content'] for result in context_results]
        
        # Generate answer with rationale
        answer_output = self.generate(context=context, question=question)
        
        # Perform self-reflection and adjust if necessary
        detailed_answer = self.add_technical_details(answer_output.answer, context)
        final_answer = self.reflect_and_adjust(detailed_answer, context, question)
        
        return final_answer

    def add_technical_details(self, answer, context):
        # Enhance the response with additional technical details
        detailed_answer = f"Technical details on the topic:\n{answer}\n"
        for additional_context in context:
            detailed_answer += f"\nAdditional Context: {additional_context}\n"
        return detailed_answer

    def reflect_and_adjust(self, answer, context, question):
        # Evaluate the response for relevance and technical accuracy
        relevance_score = self.evaluate_response(answer, context)
        if relevance_score < 0.75:  # Adjust threshold as needed
            print(f"Adjusting response for query '{question}' due to low relevance score.")
            # Logic to adjust and improve the response
            answer = self.improve_response(answer, context, question)
        return answer

    def evaluate_response(self, response, context):
        # Custom evaluation logic to score the relevance and accuracy of the response
        # For example, checking for specific keywords, coherence, or using a trained model
        score = 0.8  # Placeholder for actual evaluation logic
        return score

    def improve_response(self, response, context, question):
        # Custom logic to adjust the response for better technical accuracy
        improved_response = f"Improved technical answer to the query '{question}':\n{response}\n"
        # Integrate more context or specific examples
        for additional_context in context:
            improved_response += f"\nFurther Context: {additional_context}\n"
        return improved_response

In [30]:

# Setup WeaviateRM retriever model
retriever_model = WeaviateRM("TaxCode", weaviate_client=weaviate_client, k=10)

# Initialize RAG system
tax_rag = TaxCodeRAG(retriever_model=retriever_model)

# Example usage
question = "What are the tax implications of business expenses?"
answer = tax_rag(question=question)
print(f"Question: {question}")
print(f"Answer: {answer}")

Question: What are the tax implications of business expenses?
Answer: Technical details on the topic:
The tax implications of business expenses are that they can generally be deducted from gross income, reducing the amount of taxable income. However, businesses must keep accurate records and adhere to IRS guidelines regarding what constitutes a valid deduction. Business losses can also be deducted, but only up to certain limits and subject to specific rules depending on the type of loss and the business's overall tax situation.

Additional Context: respect to the
transaction only if the present value of the reasonably expected pre-tax profit from the
transaction is substantial in relation to the present value of the expected net tax benefits that
would be allowed if the transaction were respected.
(B) Treatment of fees and foreign taxes
Fees and other transaction expenses shall be taken into account as expenses in determining
pre-tax profit under subparagraph (A). The Secretary shall i

In [27]:
question = 'What is your tax strategy for my IT single memmber LLC Business taxed as an scorp  with gross_income of 145k and expenses around 35k and salary of 30k?'
answer = tax_rag(question=question)
print(f"Question: {question}")
print(f"Answer: {answer}")

Question: What is your tax strategy for my IT single memmber LLC Business taxed as an scorp  with gross_income of 145k and expenses around 35k and salary of 30k?
Answer: Prediction(
    rationale='determine the appropriate tax strategy for your IT single-member LLC business taxed as an S Corp with a gross income of $145k, expenses of $35k, and a salary of $30k:\n\n1. Determine the net profit: Subtract the expenses from the gross income - $145k - $35k = $110k.\n2. Deduct the salary: Since you are the sole owner and employee of the LLC, you can deduct your salary as an expense - $110k - $30k = $80k.\n3. Calculate the taxable income: Subtract the deductible expenses (including the salary) from the net profit - $80k - $35k = $45k.\n4. Apply the S Corp tax rate:',
    answer="Based on your provided information, here's a possible tax strategy for your IT single-member LLC business taxed as an S Corp:\n\n1. Determine the net profit: Subtract the expenses from the gross income - $145k - $35k =

In [28]:
question = 'What is your tax strategy for my IT single memmber LLC Business taxed as an scorp  with gross_income of 145k and expenses around 35k and salary of 30k? The answer should be in full low level detail and give me answers on what type of expenses I can deduce from my business single member llc taxed as scrop with strategies?'
answer = tax_rag(question=question)
print(f"Question: {question}")
print(f"Answer: {answer}")

Question: What is your tax strategy for my IT single memmber LLC Business taxed as an scorp  with gross_income of 145k and expenses around 35k and salary of 30k? The answer should be in full low level detail and give me answers on what type of expenses I can deduce from my business single member llc taxed as scrop with strategies?
Answer: Prediction(
    rationale="determine the best tax strategy for your IT single-member LLC business taxed as an S Corp with a gross income of $145k and expenses around $35k, and a salary of $30k.\n\n1. Deducting Business Expenses:\nAs an S Corp, you can deduct various business expenses from your net income to reduce your taxable income. Some common deductions include:\n\n- Salaries and Wages: You've already mentioned paying yourself a salary of $30k. As the owner of an S Corp, you are considered an employee, so this is a valid deduction.\n  \n- Business Interest Expense: If your LLC incurred interest expenses related to business operations or borrowing 